# 💬 Claimify reproduction study

In [1]:
# autoreload imports
%load_ext autoreload
%autoreload 2

In [2]:
from utils.chat import load_md_prompts

PROMPT_TEMPLATES = load_md_prompts("claimify")

## 1. Input Question and Answer

In [4]:
# basic chatprompttemplate llm chat with gpt-5 using 'selection' prompt template
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama

import ssl
import certifi

chat = ChatOpenAI(model="gpt-5")
# different chat (qwen:14b using ollama)
# chat = ChatOllama(model="qwen3:14b")
chat.invoke(PROMPT_TEMPLATES["selection"].format_messages(excerpt="todo", question="todo", sentence="todo"))

AIMessage(content='Sentence: \ntodo \n\n4-step stream of consciousness thought process (1. reflect on criteria at a high-level -> 2. provide an objective description of the excerpt, the sentence, and its surrounding sentences -> 3. consider all possible perspectives on whether the sentence explicitly or implicitly contains a specific and verifiable proposition, or if it just contains an introduction for the following \nsentence(s), a conclusion for the preceding sentence(s), broad or generic statements, opinions, interpretations, speculations, statements about a lack of information, etc. -> 4. only if it contains a specific and verifiable proposition: reflect on whether any changes are needed to ensure that the entire sentence only contains verifiable information):\n1) A sentence contains a specific and verifiable proposition if it asserts a concrete fact that could be checked; placeholders or generic exhortations do not qualify. \n2) The provided question, excerpt, and sentence are al

In [6]:
chat = ChatOllama(model="qwen3:14b")
chat.invoke(PROMPT_TEMPLATES["selection"].format_messages(excerpt="todo", question="todo", sentence="todo"))

AIMessage(content='<think>\nOkay, let\'s tackle this. First, I need to figure out if the given sentence contains a specific and verifiable proposition. The rules say that even if it\'s not true or relevant, as long as it\'s specific and verifiable, it counts. Let\'s look at the example provided.\n\nThe user hasn\'t provided the actual question, excerpt, or sentence, so I can\'t process the specific content. But based on the instructions, I need to check if the sentence has a verifiable fact. For instance, if the sentence was "John is the CEO of Company X," that\'s verifiable. But if it\'s a general statement like "Companies should be ethical," that\'s not.\n\nSince the user\'s input is "todo" for all parts, there\'s no actual data to analyze. Therefore, the answer would depend on the missing information. But following the structure, the response should be based on the provided example. However, without the actual sentence, I can\'t determine the answer. The correct approach would be to